In [1]:
# !pip install opencv-python
# !pip install pyzbar
# !pip install playsound==1.2.2

In [2]:
import numpy as np
import pandas as pd

import folium
import webbrowser

import matplotlib
import matplotlib.pyplot as plt

import cv2
import pyzbar.pyzbar as pyzbar
from playsound import playsound

In [3]:
def get_barcode():
    
    product_barcodes = []

    used_codes = []
    data_list = []

    try:
        f = open("qrbarcode_data.txt", "r", encoding="utf8")
        data_list = f.readlines()
    except FileNotFoundError:
        pass
    else:
        f.close()

    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)

    for i in data_list:
        used_codes.append(i.rstrip('\n'))

    while True:
        success, frame = cap.read()

        for code in pyzbar.decode(frame):
            cv2.imwrite('qrbarcode_image.png', frame)
            my_code = code.data.decode('utf-8')
            if my_code not in used_codes:
                print("인식 성공 : ", my_code)
                product_barcodes.append(my_code)
                playsound("qrbarcode_beep.mp3")
                used_codes.append(my_code)

                f2 = open("qrbarcode_data.txt", "a", encoding="utf8")
                f2.write(my_code+'\n')
                f2.close()
            elif my_code in used_codes:
                print("이미 인식된 코드 입니다.!!!")
                playsound("qrbarcode_beep.mp3")
            else:
                pass

        key = cv2.waitKey(1)
        if key == 27:
            break

        cv2.imshow('QRcode Barcode Scan', frame)
        cv2.waitKey(1)
    cap.release()
    cv2.destroyAllWindows()
    return product_barcodes

In [4]:
local_data = pd.DataFrame(columns=["SGG행정구역별","행정구역별", "물류이동개수"])
local_data["SGG행정구역별"] = pd.Series([11,21,22,23,24,25,26,29,31,32,33,34,35,36,37,38,39])
local_data["행정구역별"] = pd.Series(["서울특별시","부산광역시","대구광역시","인천광역시",
                                         "광주광역시","대전광역시","울산광역시","세종특별자치시",
                                    "경기도","강원도", "충청북도","충청남도","전라북도",
                                    "전라남도","경상북도","경상남도","제주특별자치도"])

local_count={"서울특별시":0,"부산광역시":0,"대구광역시":0,"인천광역시":0,
             "광주광역시":0,"대전광역시":0,"울산광역시":0,"세종특별자치시":0,
             "경기도":0,"강원도":0, "충청북도":0,"충청남도":0,"전라북도":0,
             "전라남도":0,"경상북도":0,"경상남도":0,"제주특별자치도":0}

In [5]:
product_barcodes = get_barcode()

for barcode in product_barcodes:
    local_num = barcode[1:3]
    if local_num =='11':
        local_count["서울특별시"]+=1
    elif local_num =='21':
        local_count["부산광역시"]+=1
    elif local_num =='22':
        local_count["대구광역시"]+=1
    elif local_num =='23':
        local_count["인천광역시"]+=1
    elif local_num =='24':
        local_count["광주광역시"]+=1
    elif local_num =='25':
        local_count["대전광역시"]+=1
    elif local_num =='26':
        local_count["울산광역시"]+=1
    elif local_num =='29':
        local_count["세종특별자치시"]+=1
    elif local_num =='31':
        local_count["경기도"]+=1
    elif local_num =='32':
        local_count["강원도"]+=1
    elif local_num =='33':
        local_count["충청북도"]+=1
    elif local_num =='34':
        local_count["충청남도"]+=1
    elif local_num =='35':
        local_count["전라북도"]+=1
    elif local_num =='36':
        local_count["전라남도"]+=1
    elif local_num =='37':
        local_count["경상북도"]+=1
    elif local_num =='38':
        local_count["경상남도"]+=1
    elif local_num =='39':
        local_count["제주특별자치도"]+=1

이미 인식된 코드 입니다.!!!
이미 인식된 코드 입니다.!!!
이미 인식된 코드 입니다.!!!
이미 인식된 코드 입니다.!!!
이미 인식된 코드 입니다.!!!
이미 인식된 코드 입니다.!!!


In [ ]:
local_data.index = local_data["행정구역별"]
local_data["물류이동개수"] = pd.Series(local_count)

local_data.to_csv("local_data.csv",index=False, encoding = 'euc-kr')
local_data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
'''
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
# font_location = '/usr/share/fonts/truetype/nanum/NanumGothicOTF.ttf'
font_location = 'C:\\font\\NanumGothic.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)
plt.rc('font', family='NanumGothic')
'''
import matplotlib
matplotlib.rcParams['font.family'] ='NanumGothic'
matplotlib.rcParams['axes.unicode_minus'] =False

local_count = pd.read_csv("local_data.csv", encoding = 'euc-kr')
local_count

In [ ]:
local_count_sorted = local_count.sort_values(by=["물류이동개수"], ascending=False)
local_count_top5 = local_count_sorted[:5]
local_count_top5

In [ ]:
ratio = local_count_top5.물류이동개수
labels = local_count_top5.행정구역별
colors = ['#ff9999', '#ffc000', '#8fd9b6', '#d395d0',
          '#ff9999', '#ffc000', '#8fd9b6', '#d395d0',
          '#ff9999', '#ffc000', '#8fd9b6', '#d395d0',
          '#ff9999', '#ffc000', '#8fd9b6', '#d395d0','#ff9999']

wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}

plt.pie(ratio, labels=labels, autopct='%.1f%%', startangle=160, \
        counterclock=False, colors=colors, wedgeprops=wedgeprops)
plt.show()

In [ ]:
def main():
    
    

    #csvFile = 'Si_Do_Parcel_Distribution.csv'
    csvFile = "local_data.csv"
    SiDodf = pd.read_csv(csvFile, encoding = 'euc-kr')
    print(SiDodf)
    
    SiDo_geo = 'Si_Do_map_utf8.json'
    
    # Initialize the map at Daejeon 
    m = folium.Map(location=[36.45, 127.42], \
                   tiles="OpenStreetMap", zoom_start=8)
    
# Json map polygon 경계좌표값으로 구역별 색칠하기 
    folium.Choropleth(
        geo_data = SiDo_geo,
        name='권역별 택배 물류 이동 비율',
        data=SiDodf,
        columns=['행정구역별', '물류이동개수'],
        key_on='feature.properties.CTP_KOR_NM',
        fill_color='PuRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='권역별 택배 물류 이동 비율').add_to(m)

    # 색칠한 layer 켜고 끄기 버튼 추가
    folium.LayerControl().add_to(m)
    
    m.save('map.html')
    webbrowser.open('map.html') 
    
    folium.LayerControl().add_to(m)
    
    m.save('map.html')
    webbrowser.open('map.html') 
    



# main 함수 로딩부
if __name__ == '__main__':
    main()